# 📝 Google Forms Creator for SME Evaluation

This notebook creates Google Forms with:
- ✅ Automatic Google Drive spreadsheet integration
- ✅ Progress saving for responders
- ✅ Proper page breaks between samples
- ✅ Professional formatting

## 🚀 Quick Start:
1. Upload your JSON data file to Colab
2. Update the `PROJECT_ID` and `JSON_FILE_PATH` below
3. Run all cells
4. Get your form URLs and spreadsheet link!


## 📦 Setup and Authentication


In [1]:
# Install required packages
%pip install -qqq google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [2]:
import json
import os
from typing import List, Dict, Any, Optional
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import default
import time

In [3]:
# 🔧 CONFIGURATION - UPDATE THESE VALUES
PROJECT_ID = "black-heuristic-463515-f6"  # Your Google Cloud Project ID
BATCH_ID = 1
JSON_FILE_PATH = f"/content/evaluation_samples_batch_{BATCH_ID}.json"

FORM_TITLE = f"BEACON LLM Evaluation for the Severity Assessment"

print(f"🔧 Project ID: {PROJECT_ID}")
print(f"📂 Data file: {JSON_FILE_PATH}")
print(f"📝 Form title: {FORM_TITLE}")


🔧 Project ID: black-heuristic-463515-f6
📂 Data file: /content/evaluation_samples_batch_1.json
📝 Form title: BEACON LLM Evaluation for the Severity Assessment


In [4]:

# Set environment variables
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

# Authenticate with broader scope for Drive access
auth.authenticate_user()
creds, _ = default(scopes=[
    'https://www.googleapis.com/auth/forms.body',
    'https://www.googleapis.com/auth/forms.responses.readonly',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/spreadsheets'
])

if hasattr(creds, 'with_quota_project'):
    creds = creds.with_quota_project(PROJECT_ID)

# Build services
forms_service = build('forms', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)
sheets_service = build('sheets', 'v4', credentials=creds)

print("✅ Google Forms API connected successfully!")
print("✅ Google Drive API connected successfully!")
print("✅ Google Sheets API connected successfully!")

✅ Google Forms API connected successfully!
✅ Google Drive API connected successfully!
✅ Google Sheets API connected successfully!


## 📊 Load and Prepare Data


In [5]:
# 📂 Load your JSON data
print(f"📂 Loading data from: {JSON_FILE_PATH}")

try:
    with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    print(f"✅ Loaded {len(sample_data)} samples!")

    # Show first sample structure
    if sample_data:
        print("\n📋 Sample data structure:")
        first_sample = sample_data[0]
        for key in first_sample.keys():
            value = str(first_sample[key])[:100] + "..." if len(str(first_sample[key])) > 100 else first_sample[key]
            print(f"  • {key}: {value}")

except FileNotFoundError:
    print(f"❌ File not found: {JSON_FILE_PATH}")

print(f"\n🎯 Ready to create form with {len(sample_data)} samples")


📂 Loading data from: /content/evaluation_samples_batch_1.json
✅ Loaded 30 samples!

📋 Sample data structure:
  • sample_id: 23
  • domain: Geographic
  • factors_description: Evaluates factors affecting disease transmission potential across geographic areas. Key factors: pop...
  • article_content: Northwestern students and staff have shown concern and support for Chinese citizens in light of the ...
  • reference_score: 5
  • reference_reasoning: All three factors scored at the highest level, indicating a severe outbreak with high potential for ...
  • model_a_score: 5
  • model_a_reasoning: The combination of high population density, high regional connectivity and mobility, and concurrent ...
  • model_b_score: 5
  • model_b_reasoning: All three factors score high, indicating a severe situation. The outbreak originated in a highly pop...
  • model_a_actual: meta-llama
  • model_b_actual: Paschalidis-NOC-Lab
  • article_id: 23

🎯 Ready to create form with 30 samples


## 🚀 Create the Google Form


In [6]:
# Add instructions

# Improved format optimized for Google Forms plain text display
instructions = """
OVERVIEW
You are participating in evaluating AI-generated outbreak severity assessments for news articles about health threats.



WHAT YOU'LL REVIEW

Article: News article from HealthMap's archived reports of historical disease outbreaks

Reference Assessment: Current "gold standard" baseline created by GPT-o1 (advanced AI model)
• Includes severity score and detailed reasoning
• Used as our training reference point

Model A & Model B Assessments: Two randomized AI model outputs for comparison
• One fine-tuned model (BEACON LLM)
• One base model (Llama-3.1-8B)
• Order is randomized to prevent bias


KEY DEFINITIONS

Severity Score Scale (1-5):
1 = Very Low Risk
2 = Low Risk
3 = Moderate Risk
4 = High Risk
5 = Very High Risk

Severity Reasoning:
Detailed justification explaining the assigned score based on relevant epidemiological, clinical, and contextual factors for the specific risk domain.


YOUR EVALUATION TASKS

Task 1: Reference Validation
Purpose: Validate our "gold standard" baseline assessment
Your Role: Evaluate whether GPT-o1's risk assessment and reasoning are accurate and appropriate
Importance: Ensures our training data quality

Task 2: Model Comparison
Purpose: Compare two different AI approaches
Your Role: Determine which model provides better risk assessment and reasoning
Focus: Overall quality, not just agreement with reference

Task 3: Expert Assessment (Optional)
Purpose: Provide independent expert judgment
When to Use: When you believe all AI models significantly over- or under-estimate
Your Role: Provide your own expert severity score with justification


EVALUATION CRITERIA

Please evaluate based on:
• Scientific Accuracy: Correctness of epidemiological and medical information
• Risk Appropriateness: How well the severity score matches the described threat level
• Reasoning Quality: Completeness and quality of the justification


SUPPORT
For technical issues or questions about this evaluation, please contact: jmyang@bu.edu

Thank you for contributing your expertise to improve BEACON LLMs."""

In [7]:
# Step 1: Create basic form (title only)
form_request = {
    "info": {
        "title": f"{FORM_TITLE} - Batch {BATCH_ID}"
    }
}

form = forms_service.forms().create(body=form_request).execute()
form_id = form['formId']

print(f"✅ Form created with ID: {form_id}")

# Step 2: Add description and settings using batchUpdate
try:
    forms_service.forms().batchUpdate(
        formId=form_id,
        body={
            "requests": [
                {
                    "updateFormInfo": {
                        "info": {
                            "title": f"{FORM_TITLE} - Batch {BATCH_ID}",
                            "description": instructions
                        },
                        "updateMask": "description"
                    }
                },
                {
                    "updateSettings": {
                        "settings": {
                            "quizSettings": {
                                "isQuiz": False
                            }
                        },
                        "updateMask": "quizSettings"
                    }
                }
            ]
        }
    ).execute()

    print("✅ Form description and settings updated successfully!")

except Exception as e:
    print(f"⚠️  Error updating form info: {e}")
    print("   Form created but description may need manual addition")

✅ Form created with ID: 1X2yZlQF-s6Mxkh4NVRVEdGDFn7LKXDUJmP_HA74xuUM
✅ Form description and settings updated successfully!


In [8]:
# Step 2: Create and link Google Spreadsheet for responses
print("📊 Creating Google Spreadsheet for responses...")

# Create spreadsheet
spreadsheet_body = {
    'properties': {
        'title': f'SME Evaluation Responses - Batch {BATCH_ID}'
    }
}

spreadsheet = sheets_service.spreadsheets().create(body=spreadsheet_body).execute()
spreadsheet_id = spreadsheet['spreadsheetId']
spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}"

print(f"✅ Spreadsheet created: {spreadsheet_url}")

# Link form to spreadsheet
try:
    # Update form settings
    forms_service.forms().batchUpdate(
        formId=form_id,
        body={
            "requests": [{
                "updateSettings": {
                    "settings": {
                        "quizSettings": {
                            "isQuiz": False
                        }
                    },
                    "updateMask": "quizSettings"
                }
            }]
        }
    ).execute()

    print("✅ Form settings updated")

except Exception as e:
    print(f"⚠️  Note: Manual spreadsheet linking may be required. Error: {e}")
    print("You can manually link the spreadsheet in the Google Forms interface.")

📊 Creating Google Spreadsheet for responses...
✅ Spreadsheet created: https://docs.google.com/spreadsheets/d/1gqLMMr6A2WfhS26ar19DpBAvDPo2VSCvj9uzEaqZvVs
✅ Form settings updated


In [9]:
# Step 3: Add form content with proper page breaks and tracking fields
print("📝 Adding form content with page breaks and tracking fields...")

all_requests = []
current_index = 0

# Add SME Email field (Page 1)
all_requests.append({
    "createItem": {
        "item": {
            "title": "👤 Your Email Address (Required)",
            "description": "Please enter your email address. This helps us track your progress and allows you to save and return to complete the form later.",
            "questionItem": {
                "question": {
                    "required": True,
                    "textQuestion": {"paragraph": False}
                }
            }
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add page break after email
all_requests.append({
    "createItem": {
        "item": {
            "pageBreakItem": {}
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add introduction text after page break
all_requests.append({
    "createItem": {
        "item": {
            "title": "📋 Begin Sample Evaluation",
            "description": "You will now evaluate each sample on a separate page. Your progress is automatically saved. You can return to this form later using the same email address.",
            "textItem": {}
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add questions for each sample with tracking fields
for i, sample in enumerate(sample_data):
    sample_num = i + 1
    sample_id = sample.get('sample_id', 'Unknown')

    # Add page break before each sample (except the first)
    if i > 0:
        all_requests.append({
            "createItem": {
                "item": {
                    "pageBreakItem": {}
                },
                "location": {"index": current_index}
            }
        })
        current_index += 1

        # Add progress indicator after page break
        all_requests.append({
            "createItem": {
                "item": {
                    "title": f"📄 Sample {sample_id}",
                    "description": f"Progress: {((i)/len(sample_data)*100):.0f}% Complete\n\n",
                    "textItem": {}
                },
                "location": {"index": current_index}
            }
        })
        current_index += 1





    # Add the sample header with corrected field reference
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"📊 Sample {sample_id} - Overview",
                "description": f"🎯 Domain: {sample.get('domain', 'Unknown Domain')}\n\n🔍 Risk Factors: {sample.get('factors_description', 'Unknown Factors')}\n\n📈 Progress: {sample_num}/{len(sample_data)} samples",
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Add article content with better formatting
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"📰 Article Content - Sample {sample_id}",
                "description": f"Please read the following article carefully:\n\n---\n\n{sample.get('article_content', 'Article content')}\n\n---",
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Add Reference assessment with enhanced formatting
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"🎯 Task 1: Reference Assessment (GPT-o1) - Sample {sample_id}",
                "description": f"""📊 Severity Score: {sample.get('reference_score', 'X')}/5

💭 Severity Reasoning:
{sample.get('reference_reasoning', 'Reference reasoning')}

---
Please evaluate this reference assessment below.""",
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Reference risk score question with enhanced options
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q1. 📊 Reference Risk Score Assessment - Sample {sample_id} (Required)",
                "description": f"Is the reference severity score ({sample.get('reference_score', 'X')}/5) appropriate for this scenario?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "📉 Too low (should be higher)"},
                                {"value": "✅ Appropriate (correct level)"},
                                {"value": "📈 Too high (should be lower)"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Reference reasoning question
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q2. 💭 Reference Reasoning Quality - Sample {sample_id} (Required)",
                "description": f"Rate the accuracy and quality of the reference reasoning:\n\n{sample.get('reference_reasoning', 'Reference reasoning')}",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "❌ Poor (Major errors or gaps)"},
                                {"value": "⚠️ Average (Acceptable but could improve)"},
                                {"value": "✅ Good (Accurate and well-reasoned)"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Model comparison display with enhanced formatting
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"🆚 Task 2: Model Comparison - Sample {sample_id}",
                "description": f"""Compare these two AI model assessments:

🤖 Model A Assessment:
📊 Severity Score: {sample.get('model_a_score', 'Y')}/5
💭 Reasoning: {sample.get('model_a_reasoning', 'Model A reasoning')}

---

🤖 Model B Assessment:
📊 Severity Score: {sample.get('model_b_score', 'Z')}/5
💭 Reasoning: {sample.get('model_b_reasoning', 'Model B reasoning')}

---
Please compare these models below.""",
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Model comparison questions with enhanced options
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q3. 📊 Model Score Comparison - Sample {sample_id} (Required)",
                "description": f"Which model's severity score is more accurate?\n\n🤖 Model A: {sample.get('model_a_score', 'Y')}/5\n🤖 Model B: {sample.get('model_b_score', 'Z')}/5",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "🥇 Model A is clearly better"},
                                {"value": "🥈 Model A is slightly better"},
                                {"value": "🤝 Both are equivalent"},
                                {"value": "🥈 Model B is slightly better"},
                                {"value": "🥇 Model B is clearly better"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Model reasoning comparison
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q4. 💭 Model Reasoning Comparison - Sample {sample_id} (Required)",
                "description": f"Which model provides better reasoning and justification?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "🥇 Model A reasoning is much better"},
                                {"value": "🥈 Model A reasoning is slightly better"},
                                {"value": "🤝 Both reasoning are equivalent"},
                                {"value": "🥈 Model B reasoning is slightly better"},
                                {"value": "🥇 Model B reasoning is much better"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Expert score (optional) with enhanced formatting
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q5. 👨‍⚕️ Your Expert Assessment - Sample {sample_id} (Optional)",
                "description": "What risk score would you assign based on your expertise? (Only provide if you believe all AI assessments are significantly off)",
                "questionItem": {
                    "question": {
                        "required": False,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "1️⃣ Very low risk"},
                                {"value": "2️⃣ Low risk"},
                                {"value": "3️⃣ Moderate risk"},
                                {"value": "4️⃣ High risk"},
                                {"value": "5️⃣ Very high risk"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

    # Comments with enhanced description
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"Q6. 💬 Additional Comments - Sample {sample_id} (Optional)",
                "description": "Any critical factors missed by all assessments or additional comments about this sample?",
                "questionItem": {
                    "question": {
                        "required": False,
                        "textQuestion": {"paragraph": True}
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

# Add final page break with completion message
all_requests.append({
    "createItem": {
        "item": {
            "pageBreakItem": {}
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add completion message
all_requests.append({
    "createItem": {
        "item": {
            "title": "🎉 Evaluation Complete!",
            "description": "Thank you for completing the evaluation. Your responses have been automatically saved to our spreadsheet. You may now submit the form.",
            "textItem": {}
        },
        "location": {"index": current_index}
    }
})
current_index += 1

print(f"📝 Prepared {len(all_requests)} form items with enhanced page breaks and tracking fields")

📝 Adding form content with page breaks and tracking fields...
📝 Prepared 363 form items with enhanced page breaks and tracking fields


In [10]:
# Step 4: Add all form items in batches with better error handling
print("🚀 Adding form items...")

batch_size = 5  # Smaller batches for better reliability
success_count = 0
total_items = len(all_requests)
failed_items = []

for i in range(0, total_items, batch_size):
    batch = all_requests[i:i+batch_size]
    batch_num = (i // batch_size) + 1
    total_batches = (total_items + batch_size - 1) // batch_size

    try:
        forms_service.forms().batchUpdate(
            formId=form_id,
            body={"requests": batch}
        ).execute()
        success_count += len(batch)
        progress = (success_count / total_items) * 100
        print(f"✅ Batch {batch_num}/{total_batches} - Added {len(batch)} items (Progress: {progress:.1f}%)")

        # Small delay to avoid rate limits
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ Batch {batch_num} error: {str(e)[:100]}...")
        # Try individually with better error handling
        for j, request in enumerate(batch):
            item_num = i + j + 1
            try:
                forms_service.forms().batchUpdate(
                    formId=form_id,
                    body={"requests": [request]}
                ).execute()
                success_count += 1
                print(f"✅ Individual item {item_num}/{total_items} added")
                time.sleep(0.2)
            except Exception as eq:
                failed_items.append({
                    'item_number': item_num,
                    'request': request,
                    'error': str(eq)[:100]
                })
                print(f"❌ Individual error {item_num}: {str(eq)[:50]}...")

print(f"\n🎉 Successfully added {success_count}/{total_items} items to the form!")

if failed_items:
    print(f"⚠️  {len(failed_items)} items failed to add:")
    for item in failed_items[:5]:  # Show first 5 failures
        print(f"   Item {item['item_number']}: {item['error']}")
    if len(failed_items) > 5:
        print(f"   ... and {len(failed_items) - 5} more")

🚀 Adding form items...
✅ Batch 1/73 - Added 5 items (Progress: 1.4%)
✅ Batch 2/73 - Added 5 items (Progress: 2.8%)
✅ Batch 3/73 - Added 5 items (Progress: 4.1%)
✅ Batch 4/73 - Added 5 items (Progress: 5.5%)
✅ Batch 5/73 - Added 5 items (Progress: 6.9%)
✅ Batch 6/73 - Added 5 items (Progress: 8.3%)
✅ Batch 7/73 - Added 5 items (Progress: 9.6%)
✅ Batch 8/73 - Added 5 items (Progress: 11.0%)
✅ Batch 9/73 - Added 5 items (Progress: 12.4%)
✅ Batch 10/73 - Added 5 items (Progress: 13.8%)
✅ Batch 11/73 - Added 5 items (Progress: 15.2%)
✅ Batch 12/73 - Added 5 items (Progress: 16.5%)
✅ Batch 13/73 - Added 5 items (Progress: 17.9%)
✅ Batch 14/73 - Added 5 items (Progress: 19.3%)
✅ Batch 15/73 - Added 5 items (Progress: 20.7%)
✅ Batch 16/73 - Added 5 items (Progress: 22.0%)
✅ Batch 17/73 - Added 5 items (Progress: 23.4%)
✅ Batch 18/73 - Added 5 items (Progress: 24.8%)
✅ Batch 19/73 - Added 5 items (Progress: 26.2%)
✅ Batch 20/73 - Added 5 items (Progress: 27.5%)
✅ Batch 21/73 - Added 5 items (Pr

## 📋 Form Information and Results


In [11]:
# Step 5: Get form URLs and create comprehensive results
print("📊 Generating form information and links...")

try:
    # Get the form data to extract the correct URLs
    form_data = forms_service.forms().get(formId=form_id).execute()

    # Extract the correct URLs
    edit_url = f"https://docs.google.com/forms/d/{form_id}/edit"

    # The responderUri gives us the correct public URL
    if 'responderUri' in form_data:
        view_url = form_data['responderUri']
    else:
        # Fallback URL format
        view_url = f"https://docs.google.com/forms/d/{form_id}/viewform"

    # Responses URL - this goes to the responses tab of the edit page
    responses_url = f"https://docs.google.com/forms/d/{form_id}/edit#responses"

    print("✅ Form URLs generated successfully")

except Exception as e:
    print(f"⚠️  Using fallback URLs due to error: {e}")
    # Use fallback URLs
    edit_url = f"https://docs.google.com/forms/d/{form_id}/edit"
    view_url = f"https://docs.google.com/forms/d/{form_id}/viewform"
    responses_url = f"https://docs.google.com/forms/d/{form_id}/edit#responses"

# Comprehensive form information
form_info = {
    'form_id': form_id,
    'edit_url': edit_url,
    'view_url': view_url,
    'responses_url': responses_url,
    'spreadsheet_id': spreadsheet_id,
    'spreadsheet_url': spreadsheet_url,
    'total_samples': len(sample_data),
    'json_file_used': JSON_FILE_PATH,
    'project_id': PROJECT_ID,
    'form_title': FORM_TITLE,
    'batch_id': BATCH_ID,
    'features': {
        'progress_saving': True,
        'page_breaks': True,
        'spreadsheet_integration': True,
        'enhanced_formatting': True
    }
}

# Save to file
output_file = f'/content/form_result_batch_{BATCH_ID}.json'
with open(output_file, 'w') as f:
    json.dump(form_info, f, indent=2)

print(f"💾 Form information saved to: {output_file}")

📊 Generating form information and links...
✅ Form URLs generated successfully
💾 Form information saved to: /content/form_result_batch_1.json


In [12]:
# Step 6: Display comprehensive results
print("\n" + "="*60)
print("🎉 FORM CREATION COMPLETED SUCCESSFULLY!")
print("="*60)

print(f"\n📝 **Form Title:** {FORM_TITLE} - Batch {BATCH_ID}")
print(f"🆔 **Form ID:** {form_id}")
print(f"📊 **Total Samples:** {len(sample_data)}")

print("\n🔗 **Important Links:**")
print(f"   📝 Edit Form: {edit_url}")
print(f"   👀 Public Form (Share with SMEs): {view_url}")
print(f"   📈 View Responses: {responses_url}")
print(f"   📊 Response Spreadsheet: {spreadsheet_url}")


print("="*60)


🎉 FORM CREATION COMPLETED SUCCESSFULLY!

📝 **Form Title:** BEACON LLM Evaluation for the Severity Assessment - Batch 1
🆔 **Form ID:** 1X2yZlQF-s6Mxkh4NVRVEdGDFn7LKXDUJmP_HA74xuUM
📊 **Total Samples:** 30

🔗 **Important Links:**
   📝 Edit Form: https://docs.google.com/forms/d/1X2yZlQF-s6Mxkh4NVRVEdGDFn7LKXDUJmP_HA74xuUM/edit
   👀 Public Form (Share with SMEs): https://docs.google.com/forms/d/e/1FAIpQLSf3T4EaSVrtLHrniT_64qa6PUFtyL8d8AZrvNhFHXzZrERLRA/viewform
   📈 View Responses: https://docs.google.com/forms/d/1X2yZlQF-s6Mxkh4NVRVEdGDFn7LKXDUJmP_HA74xuUM/edit#responses
   📊 Response Spreadsheet: https://docs.google.com/spreadsheets/d/1gqLMMr6A2WfhS26ar19DpBAvDPo2VSCvj9uzEaqZvVs
